In [ ]:
!pip install --upgrade -q git+https://github.com/terrier-org/pyterrier.git#egg=python-terrier
!pip install --upgrade -q git+https://github.com/terrierteam/pyterrier_deepct.git

In [1]:
import os
import pandas as pd
import pyterrier as pt
from pyterrier_deepct import DeepCT, Toks2Text

PyTerrier 0.9.1 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



In [2]:
deepct = DeepCT()

In [3]:
df = pd.DataFrame([{"docno" : "d1", "text" :"this is a test document"}])
df

,docno,text
0,d1,this is a test document


In [4]:
deepct(df)

,docno,text,toks
0,d1,this is a test document,"{'this': 5, 'is': 1, 'test': 69, 'document': 61}"


In [7]:
dataset = pt.get_dataset("irds:vaswani")
index_loc = "./deepct_vaswani"
indexer = deepct >> Toks2Text() >> pt.IterDictIndexer(index_loc)
indexer.index(dataset.get_corpus_iter())

vaswani documents: 100%|██████████| 11429/11429 [00:28<00:00, 401.08it/s]


<org.terrier.querying.IndexRef at 0x7f9eb2d4f0b0 jclass=org/terrier/querying/IndexRef jself=<LocalRef obj=0x55f47cd255f0 at 0x7f9eb2da6190>>

In [8]:
from pyterrier.measures import *
pt.Experiment([
    pt.BatchRetrieve(pt.get_dataset("vaswani").get_index(), wmodel="BM25"),
    pt.BatchRetrieve(index_loc, wmodel="BM25"),
  ],
  dataset.get_topics(),
  dataset.get_qrels(),
  names=["BM25", "BM25_deepct"],
  eval_metrics=[MAP, RR, R@1000]
)

,name,AP,RR,R@1000
0,BM25,0.296517,0.725665,0.934607
1,BM25_deepct,0.297491,0.702555,0.936506
